In [52]:
import csv
import random
import pandas as pd
import heapq

def weighted_duration(min_val=15, max_val=600, scale=100):
    duration = int(random.expovariate(1 / scale))
    if duration < min_val:
        return min_val
    elif duration > max_val:
        return max_val
    else:
        return duration

def generate_data(num_entries):
    time = 0
    data = []
    for _ in range(num_entries):
        time += random.randint(1, 30)
        source = random.randint(0, 9)
        receiver = random.randint(10, 19)
        bandwidth = random.randint(1, 100)
        duration = weighted_duration() 
        data.append([time, source, receiver, bandwidth, duration])
    return data

num_entries = 1500

data_entries = generate_data(num_entries)

with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['time', 'source', 'receiver', 'bandwidth', 'duration'])
    writer.writerows(data_entries)

print(f'Data written to data.csv with {num_entries} entries.')


Data written to data.csv with 1500 entries.


In [53]:
G = nx.DiGraph()

G.add_nodes_from(range(30))
flag = 0
while flag == 0:
    a, b = random.sample(list(G.nodes), 2)
    if not G.has_edge(a, b):
        G.add_edge(a, b, bandwidth=random.randint(600, 1500), used_bandwidth=0)
    flag = 1
    for i in range(10):
        for j in range(10, 20):
            flag *= int(nx.has_path(G, i, j))
for (u, v, wt) in G.edges.data('weight'):
    print(f"Edge ({u}, {v}) with weight {wt}")
print(f"Total edges: {len(G.edges)}")

Edge (0, 4) with weight None
Edge (0, 8) with weight None
Edge (0, 11) with weight None
Edge (0, 2) with weight None
Edge (0, 12) with weight None
Edge (0, 14) with weight None
Edge (0, 15) with weight None
Edge (1, 24) with weight None
Edge (1, 21) with weight None
Edge (1, 19) with weight None
Edge (2, 25) with weight None
Edge (2, 22) with weight None
Edge (2, 6) with weight None
Edge (3, 26) with weight None
Edge (3, 10) with weight None
Edge (3, 7) with weight None
Edge (4, 16) with weight None
Edge (4, 22) with weight None
Edge (4, 27) with weight None
Edge (4, 1) with weight None
Edge (5, 16) with weight None
Edge (5, 24) with weight None
Edge (5, 6) with weight None
Edge (5, 9) with weight None
Edge (5, 14) with weight None
Edge (6, 29) with weight None
Edge (6, 5) with weight None
Edge (6, 14) with weight None
Edge (7, 8) with weight None
Edge (7, 12) with weight None
Edge (7, 23) with weight None
Edge (7, 25) with weight None
Edge (7, 17) with weight None
Edge (8, 24) with we

In [54]:
data = pd.read_csv('data.csv')

def update_graph(time, active_requests):
    for u, v in G.edges:
        G[u][v]['used_bandwidth'] = 0
    for req in active_requests.values():
        if time < req['end_time']:
            path = req['path']
            for i in range(len(path) - 1):
                u, v = path[i], path[i+1]
                G[u][v]['used_bandwidth'] += req['bandwidth']
    for u, v in G.edges():
        available_bw = G[u][v]['bandwidth'] - G[u][v]['used_bandwidth']
        G[u][v]['weight'] = float('inf') if available_bw <= 0 else 1 / available_bw

def find_path(source, receiver, bandwidth):
    original_weights = {}
    for u, v in G.edges():
        original_weights[(u, v)] = G[u][v]['weight']
        available_bw = G[u][v]['bandwidth'] - G[u][v]['used_bandwidth']
        G[u][v]['weight'] = float('inf') if available_bw < bandwidth else 1 / available_bw
    try:
        path = nx.shortest_path(G, source=source, target=receiver, weight='weight')
        print(f"Found path from {source} to {receiver} with required bandwidth {bandwidth}: {path}")
    except nx.NetworkXNoPath:
        path = None
        print(f"No path available from {source} to {receiver} with required bandwidth {bandwidth}")
    for (u, v), weight in original_weights.items():
        G[u][v]['weight'] = weight

    return path
active_requests = {}

max_time = data['time'].max() + data['duration'].max()
for current_time in range(1, max_time + 1):
    print(f"Processing time {current_time}")
    new_requests = data[data['time'] == current_time]
    for _, row in new_requests.iterrows():
        if row.name not in active_requests:
            path = find_path(row['source'], row['receiver'], row['bandwidth'])
            if path:
                active_requests[row.name] = {
                    'source': row['source'],
                    'receiver': row['receiver'],
                    'bandwidth': row['bandwidth'],
                    'end_time': row['time'] + row['duration'],
                    'path': path
                }
    active_requests = {k: v for k, v in active_requests.items() if v['end_time'] >= current_time}
    update_graph(current_time, active_requests)
    print(f"Active requests at time {current_time}: {list(active_requests.keys())}")


Processing time 1
Active requests at time 1: []
Processing time 2
Active requests at time 2: []
Processing time 3
Active requests at time 3: []
Processing time 4
Active requests at time 4: []
Processing time 5
Active requests at time 5: []
Processing time 6
Found path from 4 to 19 with required bandwidth 64: [4, 1, 19]
Active requests at time 6: [0]
Processing time 7
Active requests at time 7: [0]
Processing time 8
Active requests at time 8: [0]
Processing time 9
Active requests at time 9: [0]
Processing time 10
Active requests at time 10: [0]
Processing time 11
Active requests at time 11: [0]
Processing time 12
Active requests at time 12: [0]
Processing time 13
Found path from 6 to 13 with required bandwidth 68: [6, 29, 13]
Active requests at time 13: [0, 1]
Processing time 14
Found path from 6 to 15 with required bandwidth 26: [6, 14, 20, 15]
Active requests at time 14: [0, 1, 2]
Processing time 15
Active requests at time 15: [0, 1, 2]
Processing time 16
Active requests at time 16: [